## Curate Images

This section of the code through directories of .png .jpg .jpeg or .webp files and edits/pads them in order to align eyes to save to .png with padding

### Requirements:

To run this code, you need two models in your working directory:
- `shape_predictor_5_face_landmarks.dat`: This model enables the face detection capability.
- `FSRCNN_x2.pb`: This model enables the 2x upscaling of detected faces, allowing for faster processing and good quality images.

In [6]:
import os
import cv2
import dlib
import math
import numpy as np
import random
import string
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_5_face_landmarks.dat')

def filter_images(image):
    print('filtering images')
    if image is not None and image.shape[0] > 211 and image.shape[1] > 211:
        faces = detector(cv2.cvtColor(image, cv2.COLOR_BGR2GRAY), upsample_num_times=0)
        return faces if len(faces) > 0 else None
    return None



def find_eyes(image, face):
    print('finding eyes')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    shape = predictor(gray, face)
    features = []
    for i in range(0, 5):
        features.append((i, (shape.part(i).x, shape.part(i).y)))
    return (int(features[3][1][0] + features[2][1][0]) // 2, int(features[3][1][1] + features[2][1][1]) // 2), \
           (int(features[1][1][0] + features[0][1][0]) // 2, int(features[1][1][1] + features[0][1][1]) // 2)

def upscale_image_fsrcnn(image, model_path):
    print('upscalling from fsrcnn')
    # Load the pre-trained FSRCNN_x2 model
    sr = cv2.dnn_superres.DnnSuperResImpl_create()
    sr.readModel(model_path)
    # Set the model name and scale factor
    sr.setModel("fsrcnn", 2)
    # Upscale the image
    upscaled_image = sr.upsample(image)
    return upscaled_image

def center_eyes(img, left_eye, right_eye, model_path="FSRCNN_x2.pb"):
    print('centering eyes')
    center = ((left_eye[0] + right_eye[0]) // 2, (left_eye[1] + right_eye[1]) // 2)
    dy = right_eye[1] - left_eye[1]
    dx = right_eye[0] - left_eye[0]
    angle = math.degrees(math.atan2(dy, dx))
    height, width, _ = img.shape
    eye_distance = int(math.sqrt((right_eye[0] - left_eye[0]) ** 2 + (right_eye[1] - left_eye[1]) ** 2))

    # Add border to the original image with increased blur
    padding = 512
    blur_radius = 125
    blur_radius2 = 85
    blurred_img = cv2.GaussianBlur(img, (blur_radius, blur_radius2), 0)
    padded_blurred_img = cv2.copyMakeBorder(blurred_img, padding, padding, padding, padding, cv2.BORDER_REFLECT_101)
    padded_img = cv2.copyMakeBorder(img, padding, padding, padding, padding, cv2.BORDER_REFLECT_101)

    alpha = np.zeros((height + 2 * padding, width + 2 * padding, 1), dtype=np.float32)
    alpha[padding:height + padding, padding:width + padding] = 1
    alpha_3channels = np.repeat(alpha, 3, axis=-1)  # Broadcast alpha to 3 channels
    alpha_3channels = alpha_3channels.astype(np.float64)  # Convert alpha to the same type as input images
    
    # Combine the original image and the blurred image using the alpha channel
    blended_img = cv2.multiply(padded_img.astype(np.float64), alpha_3channels) + cv2.multiply(padded_blurred_img.astype(np.float64), 1 - alpha_3channels)
    blended_img = blended_img.astype(np.uint8)

    # Update the center point after adding the border
    center = (center[0] + padding, center[1] + padding)

    M_rotate = cv2.getRotationMatrix2D(center, angle, 1)
    rotated_img = cv2.warpAffine(blended_img, M_rotate, (width + 2 * padding, height + 2 * padding), flags=cv2.INTER_LANCZOS4)

    rotated_center = tuple(map(int, M_rotate.dot(np.array([center[0], center[1], 1]))))

    dx_translation = (width + 2 * padding) // 2 - rotated_center[0]
    dy_translation = (height + 2 * padding) // 2 - rotated_center[1]
    M_translate = np.float32([[1, 0, dx_translation], [0, 1, dy_translation]])
    translated_img = cv2.warpAffine(rotated_img, M_translate, (width + 2 * padding, height + 2 * padding), flags=cv2.INTER_LANCZOS4)

    reference_eye_distance = 260
    scaling_factor = reference_eye_distance / eye_distance
    #print("eye distance",eye_distance)
    # Resize the image based on the scaling factor
    padded_height, padded_width, _ = translated_img.shape
    scaled_height = int(padded_height * scaling_factor)
    scaled_width = int(padded_width * scaling_factor)

    if scaling_factor > 1:
        try:
            translated_img = upscale_image_fsrcnn(translated_img, model_path)
        except cv2.error as e:
            print(f"Error while upscaling image: {e}")
            return None  # return None if upscaling fails
    if translated_img is not None:
        scaled_img = cv2.resize(translated_img, (scaled_width, scaled_height), interpolation=cv2.INTER_LANCZOS4)
        # Extract the central 1024x1024 block
        center_y = scaled_height // 2
        center_x = scaled_width // 2
        cropped_img = scaled_img[center_y - 512:center_y + 512, center_x - 512:center_x + 512]
        return cropped_img
    return None  # return None if translated_img is None

def crop_face(image, face):
    print('cropping face')
    left_eye, right_eye = find_eyes(image, face)
    return center_eyes(image, left_eye, right_eye)

def convert_to_png(image, faces, output_path):
    print('converting to png')
    for face in faces:
        cropped_face = crop_face(image, face)
        if cropped_face is None or cropped_face.shape[0] < 212 or cropped_face.shape[1] < 212:
            continue  # skip this face if cropping fails or the cropped image is too small
        face_area = (face.bottom() - face.top()) * (face.right() - face.left())
        random_suffix = ''.join(random.choices(string.ascii_letters, k=4))
        file_name = f'{face_area}_{random_suffix}.png'
        output_image_path = os.path.join(output_path, file_name)
        cv2.imwrite(output_image_path, cropped_face)


def process_single_image(input_image_path, output_dir):
    print('processing single image')
    image = cv2.imread(input_image_path)
    faces = filter_images(image)
    if faces is not None:
        relative_path = os.path.relpath(os.path.dirname(input_image_path), input_dir)
        output_subdir = os.path.join(output_dir, relative_path)
        os.makedirs(output_subdir, exist_ok=True)
        # Check if at least one face is greater than or equal to 512x512.
        if any((face.right() - face.left()) * (face.bottom() - face.top()) >= 212 * 212 for face in faces):
            convert_to_png(image, faces, output_subdir)



def process_images(input_dir, output_dir):
    valid_extensions = {'.png', '.jpg', '.jpeg', '.webp'}
    image_paths = [os.path.join(root, file) for root, _, files in os.walk(input_dir) for file in files
                   if os.path.splitext(file.lower())[1] in valid_extensions and
                   not any(x in file.lower() for x in ["instagram", "scan", "magazine"])]

    for image_path in image_paths:
        process_single_image(image_path, output_dir)


input_dir = 'images_uncropped'
output_dir = 'faces-cropped'
process_images(input_dir, output_dir)


processing single image
filtering images
converting to png
cropping face
finding eyes
centering eyes
upscalling from fsrcnn
processing single image
filtering images
converting to png
cropping face
finding eyes
centering eyes
upscalling from fsrcnn
